<a href="https://colab.research.google.com/github/franz6ko/natural-lenguage-processing/blob/master/challenge_3_custom_embedding_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente trabajo consiste en el entrenamiento de embeddings customizados para el análisis de las canciones escritas por diferentes artistas.

En primer lugar, se realzia el análsis para un artista en particular (Radiohead) y luego se realiza el ańalisis para varios artistas y se compara con qué relaciona cada uno el concepto de amor en sus canciones.

Para ello, se utiliza la biblioteca Gensim.

### Dependencias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import gdown
import multiprocessing

from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

import plotly.graph_objects as go
import plotly.express as px

from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                                                  

### Dataset

In [2]:
# Descarga del dataset de bandas
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('simpsons_dataset.zip', os.F_OK) is False:
        url = 'https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download'
        output = 'songs_dataset.zip'
        gdown.download(url, output, quiet=False)
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=1VLVgb3fD02XrF2V5OruKvQTOT91juXcs&export=download
To: /content/songs_dataset.zip
100%|██████████| 2.09M/2.09M [00:00<00:00, 100MB/s]


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['janisjoplin.txt',
 'johnny-cash.txt',
 'al-green.txt',
 'patti-smith.txt',
 'lorde.txt',
 'britney-spears.txt',
 'michael-jackson.txt',
 'bruce-springsteen.txt',
 'bjork.txt',
 'bob-dylan.txt',
 'amy-winehouse.txt',
 'notorious_big.txt',
 'prince.txt',
 'bruno-mars.txt',
 'blink-182.txt',
 'bieber.txt',
 'paul-simon.txt',
 'eminem.txt',
 'Kanye_West.txt',
 'ludacris.txt',
 'adele.txt',
 'drake.txt',
 'kanye.txt',
 'radiohead.txt',
 'nicki-minaj.txt',
 'nirvana.txt',
 'cake.txt',
 'beatles.txt',
 'dr-seuss.txt',
 'notorious-big.txt',
 'dj-khaled.txt',
 'lin-manuel-miranda.txt',
 'missy-elliott.txt',
 'kanye-west.txt',
 'alicia-keys.txt',
 'nickelback.txt',
 'leonard-cohen.txt',
 'Lil_Wayne.txt',
 'dickinson.txt',
 'nursery_rhymes.txt',
 'jimi-hendrix.txt',
 'bob-marley.txt',
 'lady-gaga.txt',
 'rihanna.txt',
 'r-kelly.txt',
 'joni-mitchell.txt',
 'disney.txt',
 'dolly-parton.txt',
 'lil-wayne.txt']

In [163]:
# Selección de banda y armado del dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/radiohead.txt', sep='/n', header=None, engine='python')
df.head()

,0
0,"Come on, come on"
1,You think you drive me crazy
2,"Come on, come on"
3,You and whose army?
4,You and your cronies


### Tokenización

In [164]:
sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

### Definición del modelo

In [173]:
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=500,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

w2v_model.build_vocab(sentence_tokens)

print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de docs en el corpus: 2343
Cantidad de words distintas en el corpus: 383


### Entrenamiento del modelo

##### Callback para entrenamiento

In [166]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

##### Entrenamiento

In [174]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()])

Loss after epoch 0: 94211.28125
Loss after epoch 1: 40945.046875
Loss after epoch 2: 38683.5
Loss after epoch 3: 37783.421875
Loss after epoch 4: 37884.015625
Loss after epoch 5: 37650.328125
Loss after epoch 6: 37409.15625
Loss after epoch 7: 37720.59375
Loss after epoch 8: 36621.875
Loss after epoch 9: 36807.46875
Loss after epoch 10: 36907.5
Loss after epoch 11: 35892.03125
Loss after epoch 12: 34883.96875
Loss after epoch 13: 34886.1875
Loss after epoch 14: 33710.5
Loss after epoch 15: 33604.5625
Loss after epoch 16: 32046.5
Loss after epoch 17: 31435.0625
Loss after epoch 18: 31102.375
Loss after epoch 19: 30024.9375
Loss after epoch 20: 29634.375
Loss after epoch 21: 29246.9375
Loss after epoch 22: 28999.6875
Loss after epoch 23: 27968.5625
Loss after epoch 24: 27699.9375
Loss after epoch 25: 26688.9375
Loss after epoch 26: 27023.25
Loss after epoch 27: 26490.0625
Loss after epoch 28: 25981.0625
Loss after epoch 29: 26654.3125
Loss after epoch 30: 23419.6875
Loss after epoch 31: 

(292489, 590150)

### Ensayar

In [175]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["high"], topn=10)

[('dry', 0.9795689582824707),
 ('leave', 0.9164912104606628),
 ('through', 0.9103859663009644),
 ("don't", 0.847144603729248),
 ('arms', 0.8279566764831543),
 ('blow', 0.8195592164993286),
 ('bit', 0.8118125200271606),
 ('hurt', 0.8114244937896729),
 ('reach', 0.793999969959259),
 ('listen', 0.7932729721069336)]

In [176]:
# Palabras que MAS se relacionan con...:
w2v_model.wv.most_similar(positive=["creep"], topn=10)

[('weirdo', 0.9912425875663757),
 ('ready', 0.932835578918457),
 ('stuffed', 0.8118536472320557),
 ('hole', 0.7754425406455994),
 ('but', 0.7732406854629517),
 ('such', 0.7507467269897461),
 ("i'm", 0.7470439672470093),
 ('bunker', 0.7422137260437012),
 ('not', 0.7373446226119995),
 ("who's", 0.7308545708656311)]

### Visualizar agrupación de vectores

#### Reducción de dimensionalidad con TSNE

In [177]:
def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

#### Gráfico

In [178]:
# Graficar los embedddings en 2D
x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Análisis

En el apartado de ensayos se puede apreciar la cercanía en el espacio de embeddings entre las palabras "High" y "Dry" (de la famosa canción titulada "High and Dry") y la cercanía entre las palabras "Creep" y "Weirdo" (de la famosa canción "Creep" cuyo verso canta "But I'm a creep, I'm a weirdo...")

Del gráfico podemos sacar varias conclusiones interesantes:

Las palabras "broken", "hearts" y "rain" están curiosamente muy muy próximas con lo cuál puede sospecharse que Radiohead asocia la lluvia con corazones rotos.

Las palabras "really", "messed", "up" estan muy juntas también. Parecería ser un tema recurrente en sus letras.

### Comparación entre artistas

#### Pipeline

In [185]:
artists = ["amy-winehouse", "bruno-mars", "eminem", "adele", "blink-182"]

models = dict()

for artist in artists:
  dfx = pd.read_csv('songs_dataset/' + artist + '.txt', sep='/n', header=None, engine='python')

  sentence_tokensx = []
  for _, row in dfx[:None].iterrows():
      sentence_tokensx.append(text_to_word_sequence(row[0]))

  w2v_modelx = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                      window=2,       # cant de palabras antes y desp de la predicha
                      size=600,       # dimensionalidad de los vectores 
                      negative=20,    # cantidad de negative samples... 0 es no se usa
                      workers=1,      # si tienen más cores pueden cambiar este valor
                      sg=1)           # modelo 0:CBOW  1:skipgram

  w2v_modelx.build_vocab(sentence_tokensx)

  w2v_modelx.train(sentence_tokensx,
                  total_examples=w2v_modelx.corpus_count,
                  epochs=30)
  
  models[artist] = w2v_modelx

#### Ensayos

In [183]:
models['amy-winehouse'].wv.most_similar(positive=["love"], topn=10)

[('tomorrow', 0.8980942964553833),
 ('tonight', 0.8919981718063354),
 ('losing', 0.8649455308914185),
 ('still', 0.853580117225647),
 ('is', 0.815991997718811),
 ('sure', 0.8095980882644653),
 ('safe', 0.7633935213088989),
 ('let', 0.7545422911643982),
 ('made', 0.7448526620864868),
 ('music', 0.7437902092933655)]

In [184]:
models['bruno-mars'].wv.most_similar(positive=["love"], topn=10)

[('gorillas', 0.6104142665863037),
 ('makes', 0.5742641687393188),
 ('madly', 0.5632285475730896),
 ('yes', 0.5507463216781616),
 ('had', 0.5363439917564392),
 ('faith', 0.5293291807174683),
 ('seems', 0.5271198153495789),
 ('sing', 0.5255814790725708),
 ('promise', 0.522048830986023),
 ('making', 0.5112415552139282)]

In [186]:
models['eminem'].wv.most_similar(positive=["love"], topn=10)

[('ladies', 0.6239629983901978),
 ('hate', 0.5897095203399658),
 ('showed', 0.5820305347442627),
 ('em', 0.5795252323150635),
 ('toe', 0.5684994459152222),
 ('crying', 0.5592947602272034),
 ('dreams', 0.5585983395576477),
 ('because', 0.557192325592041),
 ('nick', 0.5526882410049438),
 ('favors', 0.548558235168457)]

In [187]:
models['adele'].wv.most_similar(positive=["love"], topn=10)

[("that's", 0.7391064167022705),
 ('forgive', 0.7234510779380798),
 ('cry', 0.683314323425293),
 ('until', 0.6710277795791626),
 ('first', 0.6708269119262695),
 ('lover', 0.6638697981834412),
 ('crazy', 0.6452536582946777),
 ('send', 0.6396065950393677),
 ('god', 0.6391588449478149),
 ('thing', 0.6374861001968384)]

In [188]:
models['blink-182'].wv.most_similar(positive=["love"], topn=10)

[('falls', 0.8442625999450684),
 ('rain', 0.7714488506317139),
 ('fell', 0.7648094892501831),
 ('deep', 0.7317363619804382),
 ('sneak', 0.7312573194503784),
 ('those', 0.7291802167892456),
 ('background', 0.7224054336547852),
 ('house', 0.6976962089538574),
 ('radio', 0.6943206787109375),
 ('guy', 0.6940866708755493)]

### Análisis comparativo

Analizando las celdas anteriores, podemos darnos una idea del concepto de amor que tienen los artistas elegidos y con qué lo suelen relacionar.



*   Amy parece relacionar el amor con seguridad (sure, safe) y con algo constante de todos los días (still, tonight, tomorrow)
*   Bruno parece tener una mirada del amor más orientada al deseo y lo sexual (makes/making love, madly). Como dice su canción "we'll be making love like gorillaz)
*   Eminem en sus letras lo asocia con dolor y con mujeres (ladies, crying, hate)
*   Adele, con muchas emociones diferentes (forgive, cry, crazy)
*   Blink habla sobre enamorarse (falls/fell in love) y lo asocia con algo profundo y con la lluvia

